In [5]:
import tensorflow as tf
import numpy
import random
import math

In [14]:
%%latex 

\documentclass{article}
\usepackage{tikz}
\usetikzlibrary{matrix,chains,positioning,decorations.pathreplacing,arrows}

\begin{document}

\begin{tikzpicture}[
plain/.style={
  draw=none,
  fill=none,
  },
net/.style={
  matrix of nodes,
  nodes={
    draw,
    circle,
    inner sep=10pt
    },
  nodes in empty cells,
  column sep=2cm,
  row sep=-9pt
  },
>=latex
]
\matrix[net] (mat)
{
|[plain]| \parbox{1.3cm}{\centering Input\\layer} & |[plain]| \parbox{1.3cm}{\centering Hidden\\layer} & |[plain]| \parbox{1.3cm}{\centering Output\\layer} \    & |[plain]| \    |[plain]| & \    & |[plain]| \    |[plain]| & |[plain]| \    & & \    |[plain]| & |[plain]| \    & |[plain]| \    |[plain]| & \    & |[plain]| \    };
\foreach \ai [count=\mi ]in {2,4,...,10}
  \draw[<-] (mat-\ai-1) -- node[above] {Input \mi} +(-2cm,0);
\foreach \ai in {2,4,...,10}
{\foreach \aii in {3,6,9}
  \draw[->] (mat-\ai-1) -- (mat-\aii-2);
}
\foreach \ai in {3,6,9}
  \draw[->] (mat-\ai-2) -- (mat-6-3);
\draw[->] (mat-6-3) -- node[above] {Ouput} +(2cm,0);
\end{tikzpicture}

\begin{tikzpicture}[
init/.style={
  draw,
  circle,
  inner sep=2pt,
  font=\Huge,
  join = by -latex
},
squa/.style={
  draw,
  inner sep=2pt,
  font=\Large,
  join = by -latex
},
start chain=2,node distance=13mm
]
\node[on chain=2] 
  (x2) {$x_2$};
\node[on chain=2,join=by o-latex] 
  {$w_2$};
\node[on chain=2,init] (sigma) 
  {$\displaystyle\Sigma$};
\node[on chain=2,squa,label=above:{\parbox{2cm}{\centering Activate \\ function}}]   
  {$f$};
\node[on chain=2,label=above:Output,join=by -latex] 
  {$y$};
\begin{scope}[start chain=1]
\node[on chain=1] at (0,1.5cm) 
  (x1) {$x_1$};
\node[on chain=1,join=by o-latex] 
  (w1) {$w_1$};
\end{scope}
\begin{scope}[start chain=3]
\node[on chain=3] at (0,-1.5cm) 
  (x3) {$x_3$};
\node[on chain=3,label=below:Weights,join=by o-latex] 
  (w3) {$w_3$};
\end{scope}
\node[label=above:\parbox{2cm}{\centering Bias \\ $b$}] at (sigma|-w1) (b) {};

\draw[-latex] (w1) -- (sigma);
\draw[-latex] (w3) -- (sigma);
\draw[o-latex] (b) -- (sigma);

\draw[decorate,decoration={brace,mirror}] (x1.north west) -- node[left=10pt] {Inputs} (x3.south west);
\end{tikzpicture}

\end{document}

<IPython.core.display.Latex object>

In [175]:
# tf Graph Input
num_inputs = 32
num_hidden_1 = 256
num_hidden_2 = 256
num_outputs = 32

inputs = tf.placeholder(dtype=tf.float32, shape=[None, num_inputs], name="inputs")
expected = tf.placeholder(dtype=tf.float32, shape=[None, num_outputs], name="expected")

# First hidden layer of the network 
h1w = tf.Variable(expected_shape=[num_inputs, num_hidden_1], name="hidden_1_weight", 
                 initial_value=tf.truncated_normal([num_inputs, num_hidden_1], stddev=0.25))
h1b = tf.Variable(expected_shape=[1, num_hidden_1], name="hidden_1_bias", 
                 initial_value=tf.truncated_normal([1, num_hidden_1], stddev=0.25))

h1l = tf.nn.sigmoid(tf.add(tf.matmul(inputs, h1w), h1b))

# Second hidden layer of the network 
h2w = tf.Variable(expected_shape=[num_hidden_1, num_hidden_2], name="hidden_2_weight", 
                 initial_value=tf.truncated_normal([num_hidden_1, num_hidden_2], stddev=0.25))
h2b = tf.Variable(expected_shape=[1, num_hidden_2], name="hidden_2_bias", 
                 initial_value=tf.truncated_normal([1, num_hidden_2], stddev=0.25))

h2l = tf.nn.sigmoid(tf.add(tf.matmul(h1l, h2w), h2b))

# Weight and biases used to turn hidden layer into final output value. 
ow = tf.Variable(expected_shape=[num_hidden_2, num_outputs], name="output_weight", 
                 initial_value=tf.truncated_normal([num_hidden_2, num_outputs], stddev=0.25))
ob = tf.Variable(expected_shape=[1, num_outputs], name="output_bias", 
                 initial_value=tf.truncated_normal([1, num_outputs], stddev=0.25))

output = tf.add(tf.matmul(h2l, ow), ob)
final = tf.argmax(tf.nn.softmax(output),1)

# We compute error as the mean square difference 
error = tf.reduce_mean(tf.pow(tf.subtract(output, expected), 2))

In [176]:
# Parameters
learning_rate = 0.0025

# Gradient descent
# optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(error)
# optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(error)

In [177]:
training_steps = 25000

min = 1
max = 11

display_step = 500
batch_size = 123


def binlist(x, length):
    return [1.0 * (0!=(x&(1<<b))) for b in range(length)]

def onehot(x, length):
    return [1.0 * (x==b) for b in range(length)]

def lesshot(x, length):
    return [1.0 * (x<=b) for b in range(length)]

def zero(length): return [0] * length 

def makeinput(x,y):
    return [x, math.log(x)] + onehot(x, 14) + [y, math.log(y)] + onehot(y, 14)

def f(x,y):  
    return onehot(x*y, 32)

# Launch the graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_steps):
        data = { inputs: [], expected: []}
        for b in range(batch_size):
#            x = random.uniform(min,max)
#            y = random.uniform(min,max)
            x = random.randint(min,max)
            y = random.randint(min,max)
            data[inputs].append(makeinput(x,y))
            data[expected].append(f(x,y))
        sess.run(optimizer, feed_dict=data)

        if epoch % display_step == 0:
#           print "data=", data 
            e = sess.run(error, feed_dict=data)
            print "Epoch:", epoch, " error=", e
#    print "iw= ", sess.run(h1w)
#    print "ib= ", sess.run(h1b)
#    print "ow= ", sess.run(ow)
#    print "ob= ", sess.run(ob)
    for x in range(min, max):
        for y in range(min, max):
            data = {inputs: [makeinput(x,y)], expected: [f(x,y)]}
            print " x=", x, " y=", y, " final=", sess.run(final, feed_dict=data), " err=",  sess.run(error, feed_dict=data)

Epoch: 0  error= 4.77259
Epoch: 500  error= 0.0349073
Epoch: 1000  error= 0.0294826
Epoch: 1500  error= 0.0283848
Epoch: 2000  error= 0.0250567
Epoch: 2500  error= 0.0259698
Epoch: 3000  error= 0.0279335
Epoch: 3500  error= 0.0249026
Epoch: 4000  error= 0.0185042
Epoch: 4500  error= 0.0217147
Epoch: 5000  error= 0.0186127
Epoch: 5500  error= 0.0204421
Epoch: 6000  error= 0.0188873
Epoch: 6500  error= 0.0203958
Epoch: 7000  error= 0.0215895
Epoch: 7500  error= 0.0201102
Epoch: 8000  error= 0.0181754
Epoch: 8500  error= 0.0196443
Epoch: 9000  error= 0.0195766
Epoch: 9500  error= 0.0194217
Epoch: 10000  error= 0.020162
Epoch: 10500  error= 0.0211055
Epoch: 11000  error= 0.021412
Epoch: 11500  error= 0.0204951
Epoch: 12000  error= 0.0198731
Epoch: 12500  error= 0.0193826
Epoch: 13000  error= 0.0209889
Epoch: 13500  error= 0.0202145
Epoch: 14000  error= 0.0200615
Epoch: 14500  error= 0.0202297
Epoch: 15000  error= 0.0188876
Epoch: 15500  error= 0.022628
Epoch: 16000  error= 0.0184934
Epoch:

In [ ]:
# Regression result